In [1]:
import numpy as np
import time
import cv2
import os
import pickle
from tqdm import tqdm
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Extract and Aggregate SIFT Feature of the Database

1. **Extract** and **Aggregate** features from all the databse images. I use SIFT here, but SURF or ORB may also work. Every step, we directly read an image from the SSD, extract featrue with help of OpenCV 4.4+, aggregate the extracted feature into a big list `database_entire_des` that would store all the features from all the images, by taking advantage of python's `extend` so we are not creating nested array.
    - It's tempting to store all the loaded images or computed (SIFT) feature and retrieve them later from RAM. But my experiment showed that it's actually faster if we just load them and re-compute (SIFT) feature whenever we need any one of these two, as SSD + CPU doing heavylifting is faster than CPU + RAM doing heavylifing. Anyway, I didn't store the computed SIFT or loaded images anywhere

In [543]:
database_path = r'./T4imgs/database/'
query_path = r"./T4imgs/queries/"

In [544]:
SIFT_extractor = cv2.SIFT_create()

In [545]:
def extract_aggregate_feature(extractor, folder_path):
    all_descriptors = list()
    database_feature = dict()
    
    for img_name in tqdm(os.listdir(folder_path)):
        
        # get feature
        img = cv2.imread(folder_path + img_name)
        kp, des = extractor.detectAndCompute(img, None)
        
        # append the descriptor to the aggregated list
        all_descriptors.extend(des)
    
    # use np.asarray so we don't copy and save some memory
    all_descriptors = np.asarray(all_descriptors)

    return all_descriptors

In [546]:
database_entire_des = extract_aggregate_feature(SIFT_extractor, database_path)

100%|████████████████████████████████████████████████████████████████████████████| 28600/28600 [06:36<00:00, 72.09it/s]


# Clustering the Features using K-mean

2. **Clustering** the entire list of features by using scikit-learn's KMeans class. 
    - We cluster them into 16 class and only run it 1 times, because in our case we are actually finding the identical image in the database given a query, and the image is very small itslef. In the real world if the query image is larger and not even in the database, so we can only find similar one, we would need more classes (64 or 256) and more run of the clustering algorithm to get the best performing one (like 10 times) so the generated codebook is more robust.
    - I turned down the text display for training so it's less verbose.

In [547]:
# clustering the entire bag of descriptor
codebook = KMeans(n_clusters = 16, init='k-means++', n_init=1, verbose=0).fit(database_entire_des)

# Compute VLAD Feature for Every Database Image

3. **Compute** VLAD feature for every image. We first load all images, compute their (SIFT) features on the fly as it's faster like I said before. We then use this feature to compute the corresponding VLAD feature of this image, and append it to the list `database_VLAD`, where each element is a VLAD feature.
    - We compute VLAD by compute the sum of residue to each centroid and concatenate these vectors.
    - We normalize the VLAD vector using square root normalization and L2 normalization.
    - My implementation of VLAD calculation is adapted from here: https://github.com/jorjasso/VLAD/blob/eeaad96c33aea9c11bceb285ba5cdabba9fb663f/VLADlib/VLAD.py#L177
4. At the same time we create a list `database_name` used to hold all the names of database image. Because we are inserting name to this list at the same time we create and append a VLAD feature, we now have a one-to-one mapping between `database_VLAD` and `database_name`, i.e. two list is pointing to the same image if given two identical index.

In [548]:
def get_VLAD(X, codebook):

    predictedLabels = codebook.predict(X)
    centroids = codebook.cluster_centers_
    labels = codebook.labels_
    k = codebook.n_clusters
   
    m,d = X.shape
    VLAD_feature = np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels == i) > 0:
            # add the diferences
            VLAD_feature[i] = np.sum(X[predictedLabels==i,:] - centroids[i],axis=0)
    

    VLAD_feature = VLAD_feature.flatten()
    # power normalization, also called square-rooting normalization
    VLAD_feature = np.sign(VLAD_feature)*np.sqrt(np.abs(VLAD_feature))

    # L2 normalization
    VLAD_feature = VLAD_feature/np.linalg.norm(VLAD_feature)
    return VLAD_feature

In [549]:
database_VLAD = list()
database_name = list()
for img_name in tqdm(os.listdir(database_path)):
    img = cv2.imread(database_path + img_name)
    kp, des = SIFT_extractor.detectAndCompute(img, None)
    VLAD = get_VLAD(des, codebook)
    database_VLAD.append(VLAD)
    database_name.append(img_name)
    
database_VLAD = np.asarray(database_VLAD)
    #database_VLAD[VLAD] = img_name

100%|████████████████████████████████████████████████████████████████████████████| 28600/28600 [07:12<00:00, 66.16it/s]


# Indexing all the VLAD Features

5. **Indexing** all the VLAD features by creating a `BallTree` of the list `database_VLAD`. This is not essential because we can also do a pair-wise comparison, but BallTree saves a lot of time when retrieving the item that has the smallest distance to the query. This is not essential so I will skip explaining Balltree. But generally, it's a efficient indexing method that performs better when the data is high dimensional, comparing to its alternative KD-Tree
    - I am using L2 distance as the measure between VLAD features. But the choice doesn't matter in this specific problem, because again, we are finding the exact same picture so the distance, no matter what, would be 0, since SIFT and VLAD are both not randomized, so for the same image they would generate the same feature vector.

In [550]:
tree = BallTree(database_VLAD, leaf_size=60)

# Query

6. Compute (SIFT) feature of all the query images, and then compute VLAD feature accordingly, using the same clustering as the database. We then get the VLAD feature of this 5 query images. We then find the images in database whose VLAD feature distance to these query images are 0, respectively.
    - Here we are finding only the top 1 match in `tree.query`, because we know the distance will be 0 as the image in query will have an identical one in the database. In the real world when finding similar images, we will need more matching like 3 or 5, and manually or use some other algorithm to further identify the most similar one

In [551]:
query_path = r"./T4imgs/queries/"

In [552]:
value_list = list()
for img_name in tqdm(os.listdir(query_path)):
    query = cv2.imread(query_path+img_name)
    q_kp, q_des = SIFT_extractor.detectAndCompute(query, None)
    query_VLAD = get_VLAD(q_des, codebook).reshape(1, -1)
    
    # we only want the cloest one
    dist, index = tree.query(query_VLAD, 1)
    
    # index is an array of array of 1
    value_name = database_name[index[0][0]]
    value_list.append(value_name)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.23it/s]


# Results

7. Below is the matches in the database for all the query images, in the order from query1.png to query5.png

In [553]:
value_list

['image1373.png',
 'image2622.png',
 'image6051.png',
 'image26588.png',
 'image13935.png']